In [1]:
!pip install autokeras


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 162 kB 4.4 MB/s 
     |████████████████████████████████| 135 kB 38.9 MB/s 
     |████████████████████████████████| 1.6 MB 36.8 MB/s 


In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf

import autokeras as ak


## A Simple Example
The first step is to prepare your data. Here we use the [Titanic
dataset](https://www.kaggle.com/c/titanic) as an example.


In [3]:

TRAIN_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/train.csv"
TEST_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/eval.csv"

train_file_path = tf.keras.utils.get_file("train.csv", TRAIN_DATA_URL)
test_file_path = tf.keras.utils.get_file("eval.csv", TEST_DATA_URL)


13049/13049 [==============================] - 0s 0us/step


The second step is to run the
[StructuredDataClassifier](/structured_data_classifier).
As a quick demo, we set epochs to 10.
You can also leave the epochs unspecified for an adaptive number of epochs.


In [4]:
# Initialize the structured data classifier.
clf = ak.StructuredDataClassifier(
    overwrite=True, max_trials=3
)  # It tries 3 different models.
# Feed the structured data classifier with training data.
clf.fit(
    # The path to the train.csv file.
    train_file_path,
    # The name of the label column.
    "survived",
    epochs=10,
)
# Predict with the best model.
predicted_y = clf.predict(test_file_path)
# Evaluate the best model with testing data.
print(clf.evaluate(test_file_path, "survived"))


Trial 3 Complete [00h 00m 04s]
val_accuracy: 0.33043476939201355

Best val_accuracy So Far: 0.8608695864677429
Total elapsed time: 00h 00m 12s
Epoch 1/10
20/20 [==============================] - 1s 3ms/step - loss: 0.6852 - accuracy: 0.5423
Epoch 2/10
20/20 [==============================] - 0s 3ms/step - loss: 0.5935 - accuracy: 0.7512
Epoch 3/10
20/20 [==============================] - 0s 4ms/step - loss: 0.5362 - accuracy: 0.7624
Epoch 4/10
20/20 [==============================] - 0s 3ms/step - loss: 0.4991 - accuracy: 0.7847
Epoch 5/10
20/20 [==============================] - 0s 3ms/step - loss: 0.4742 - accuracy: 0.8166
Epoch 6/10
20/20 [==============================] - 0s 3ms/step - loss: 0.4560 - accuracy: 0.8230
Epoch 7/10
20/20 [==============================] - 0s 4ms/step - loss: 0.4423 - accuracy: 0.8246
Epoch 8/10
20/20 [==============================] - 0s 3ms/step - loss: 0.4324 - accuracy: 0.8325
Epoch 9/10
20/20 [==============================] - 0s 3ms/step - loss: 0

## Data Format
The AutoKeras StructuredDataClassifier is quite flexible for the data format.

The example above shows how to use the CSV files directly. Besides CSV files,
it also supports numpy.ndarray, pandas.DataFrame or [tf.data.Dataset](
https://www.tensorflow.org/api_docs/python/tf/data/Dataset?version=stable). The
data should be two-dimensional with numerical or categorical values.

For the classification labels,
AutoKeras accepts both plain labels, i.e. strings or integers, and one-hot encoded
encoded labels, i.e. vectors of 0s and 1s.
The labels can be numpy.ndarray, pandas.DataFrame, or pandas.Series.

The following examples show how the data can be prepared with numpy.ndarray,
pandas.DataFrame, and tensorflow.data.Dataset.


In [5]:

# x_train as pandas.DataFrame, y_train as pandas.Series
x_train = pd.read_csv(train_file_path)
print(type(x_train))  # pandas.DataFrame
y_train = x_train.pop("survived")
print(type(y_train))  # pandas.Series

# You can also use pandas.DataFrame for y_train.
y_train = pd.DataFrame(y_train)
print(type(y_train))  # pandas.DataFrame

# You can also use numpy.ndarray for x_train and y_train.
x_train = x_train.to_numpy()
y_train = y_train.to_numpy()
print(type(x_train))  # numpy.ndarray
print(type(y_train))  # numpy.ndarray

# Preparing testing data.
x_test = pd.read_csv(test_file_path)
y_test = x_test.pop("survived")

# It tries 10 different models.
clf = ak.StructuredDataClassifier(overwrite=True, max_trials=3)
# Feed the structured data classifier with training data.
clf.fit(x_train, y_train, epochs=10)
# Predict with the best model.
predicted_y = clf.predict(x_test)
# Evaluate the best model with testing data.
print(clf.evaluate(x_test, y_test))


Trial 3 Complete [00h 00m 04s]
val_accuracy: 0.852173924446106

Best val_accuracy So Far: 0.8608695864677429
Total elapsed time: 00h 00m 12s
Epoch 1/10
20/20 [==============================] - 1s 3ms/step - loss: 0.6298 - accuracy: 0.7305
Epoch 2/10
20/20 [==============================] - 0s 4ms/step - loss: 0.5512 - accuracy: 0.7895
Epoch 3/10
20/20 [==============================] - 0s 4ms/step - loss: 0.4979 - accuracy: 0.8022
Epoch 4/10
20/20 [==============================] - 0s 4ms/step - loss: 0.4647 - accuracy: 0.8086
Epoch 5/10
20/20 [==============================] - 0s 4ms/step - loss: 0.4450 - accuracy: 0.8134
Epoch 6/10
20/20 [==============================] - 0s 3ms/step - loss: 0.4324 - accuracy: 0.8262
Epoch 7/10
20/20 [==============================] - 0s 4ms/step - loss: 0.4235 - accuracy: 0.8246
Epoch 8/10
20/20 [==============================] - 0s 4ms/step - loss: 0.4165 - accuracy: 0.8309
Epoch 9/10
20/20 [==============================] - 0s 3ms/step - loss: 0.4

The following code shows how to convert numpy.ndarray to tf.data.Dataset.


In [6]:
train_set = tf.data.Dataset.from_tensor_slices((x_train.astype(np.unicode), y_train))
test_set = tf.data.Dataset.from_tensor_slices(
    (x_test.to_numpy().astype(np.unicode), y_test)
)

clf = ak.StructuredDataClassifier(overwrite=True, max_trials=3)
# Feed the tensorflow Dataset to the classifier.
clf.fit(train_set, epochs=10)
# Predict with the best model.
predicted_y = clf.predict(test_set)
# Evaluate the best model with testing data.
print(clf.evaluate(test_set))


Trial 3 Complete [00h 00m 07s]
val_accuracy: 0.8608695864677429

Best val_accuracy So Far: 0.8695651888847351
Total elapsed time: 00h 00m 18s
Epoch 1/10
20/20 [==============================] - 1s 4ms/step - loss: 0.6095 - accuracy: 0.7656
Epoch 2/10
20/20 [==============================] - 0s 5ms/step - loss: 0.5455 - accuracy: 0.8054
Epoch 3/10
20/20 [==============================] - 0s 5ms/step - loss: 0.5015 - accuracy: 0.8182
Epoch 4/10
20/20 [==============================] - 0s 5ms/step - loss: 0.4694 - accuracy: 0.8230
Epoch 5/10
20/20 [==============================] - 0s 5ms/step - loss: 0.4476 - accuracy: 0.8262
Epoch 6/10
20/20 [==============================] - 0s 5ms/step - loss: 0.4337 - accuracy: 0.8246
Epoch 7/10
20/20 [==============================] - 0s 7ms/step - loss: 0.4245 - accuracy: 0.8246
Epoch 8/10
20/20 [==============================] - 0s 5ms/step - loss: 0.4180 - accuracy: 0.8278
Epoch 9/10
20/20 [==============================] - 0s 5ms/step - loss: 0.

You can also specify the column names and types for the data as follows.  The
`column_names` is optional if the training data already have the column names,
e.g.  pandas.DataFrame, CSV file.  Any column, whose type is not specified will
be inferred from the training data.


In [7]:
# Initialize the structured data classifier.
clf = ak.StructuredDataClassifier(
    column_names=[
        "sex",
        "age",
        "n_siblings_spouses",
        "parch",
        "fare",
        "class",
        "deck",
        "embark_town",
        "alone",
    ],
    column_types={"sex": "categorical", "fare": "numerical"},
    max_trials=10,  # It tries 10 different models.
    overwrite=True,
)



## Validation Data
By default, AutoKeras use the last 20% of training data as validation data.  As
shown in the example below, you can use `validation_split` to specify the
percentage.


In [8]:
clf.fit(
    x_train,
    y_train,
    # Split the training data and use the last 15% as validation data.
    validation_split=0.15,
    epochs=10,
)


Trial 10 Complete [00h 00m 04s]
val_accuracy: 0.9036144614219666

Best val_accuracy So Far: 0.9036144614219666
Total elapsed time: 00h 00m 42s
Epoch 1/10
20/20 [==============================] - 1s 3ms/step - loss: 0.6768 - accuracy: 0.5821
Epoch 2/10
20/20 [==============================] - 0s 3ms/step - loss: 0.6046 - accuracy: 0.7480
Epoch 3/10
20/20 [==============================] - 0s 3ms/step - loss: 0.5466 - accuracy: 0.7943
Epoch 4/10
20/20 [==============================] - 0s 3ms/step - loss: 0.5006 - accuracy: 0.8134
Epoch 5/10
20/20 [==============================] - 0s 3ms/step - loss: 0.4680 - accuracy: 0.8246
Epoch 6/10
20/20 [==============================] - 0s 3ms/step - loss: 0.4467 - accuracy: 0.8293
Epoch 7/10
20/20 [==============================] - 0s 3ms/step - loss: 0.4331 - accuracy: 0.8293
Epoch 8/10
20/20 [==============================] - 0s 3ms/step - loss: 0.4237 - accuracy: 0.8357
Epoch 9/10
20/20 [==============================] - 0s 3ms/step - loss: 0

You can also use your own validation set
instead of splitting it from the training data with `validation_data`.


In [9]:
split = 500
x_val = x_train[split:]
y_val = y_train[split:]
x_train = x_train[:split]
y_train = y_train[:split]
clf.fit(
    x_train,
    y_train,
    # Use your own validation set.
    validation_data=(x_val, y_val),
    epochs=10,
)


## Customized Search Space
For advanced users, you may customize your search space by using
[AutoModel](/auto_model/#automodel-class) instead of
[StructuredDataClassifier](/structured_data_classifier). You can configure the
[StructuredDataBlock](/block/#structureddatablock-class) for some high-level
configurations, e.g., `categorical_encoding` for whether to use the
[CategoricalToNumerical](/block/#categoricaltonumerical-class). You can also do
not specify these arguments, which would leave the different choices to be
tuned automatically. See the following example for detail.


In [10]:

input_node = ak.StructuredDataInput()
output_node = ak.StructuredDataBlock(categorical_encoding=True)(input_node)
output_node = ak.ClassificationHead()(output_node)
clf = ak.AutoModel(
    inputs=input_node, outputs=output_node, overwrite=True, max_trials=3
)
clf.fit(x_train, y_train, epochs=10)


Trial 3 Complete [00h 00m 03s]
val_loss: 0.4507189691066742

Best val_loss So Far: 0.4507189691066742
Total elapsed time: 00h 00m 10s
Epoch 1/10
16/16 [==============================] - 1s 2ms/step - loss: 4.5929 - accuracy: 0.3800
Epoch 2/10
16/16 [==============================] - 0s 2ms/step - loss: 2.6032 - accuracy: 0.4260
Epoch 3/10
16/16 [==============================] - 0s 4ms/step - loss: 2.1961 - accuracy: 0.5280
Epoch 4/10
16/16 [==============================] - 0s 3ms/step - loss: 2.0606 - accuracy: 0.5420
Epoch 5/10
16/16 [==============================] - 0s 3ms/step - loss: 1.6043 - accuracy: 0.5700
Epoch 6/10
16/16 [==============================] - 0s 3ms/step - loss: 1.6738 - accuracy: 0.5800
Epoch 7/10
16/16 [==============================] - 0s 2ms/step - loss: 1.7717 - accuracy: 0.5740
Epoch 8/10
16/16 [==============================] - 0s 3ms/step - loss: 1.5117 - accuracy: 0.6020
Epoch 9/10
16/16 [==============================] - 0s 2ms/step - loss: 1.2009 - a

The usage of [AutoModel](/auto_model/#automodel-class) is similar to the
[functional API](https://www.tensorflow.org/guide/keras/functional) of Keras.
Basically, you are building a graph, whose edges are blocks and the nodes are
intermediate outputs of blocks.
To add an edge from `input_node` to `output_node` with
`output_node = ak.[some_block]([block_args])(input_node)`.

You can even also use more fine grained blocks to customize the search space even
further. See the following example.


In [11]:

input_node = ak.StructuredDataInput()
output_node = ak.CategoricalToNumerical()(input_node)
output_node = ak.DenseBlock()(output_node)
output_node = ak.ClassificationHead()(output_node)
clf = ak.AutoModel(
    inputs=input_node, outputs=output_node, overwrite=True, max_trials=1
)
clf.fit(x_train, y_train, epochs=1)
clf.predict(x_train)


Trial 1 Complete [00h 00m 02s]
val_loss: 0.5725904107093811

Best val_loss So Far: 0.5725904107093811
Total elapsed time: 00h 00m 02s
16/16 [==============================] - 0s 2ms/step


array([[0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],

You can also export the best model found by AutoKeras as a Keras Model.


In [12]:
model = clf.export_model()
model.summary()
print(x_train.dtype)
# numpy array in object (mixed type) is not supported.
# convert it to unicode.
model.predict(x_train.astype(np.unicode))


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 9)]               0         
                                                                 
 multi_category_encoding (Mu  (None, 9)                0         
 ltiCategoryEncoding)                                            
                                                                 
 dense (Dense)               (None, 32)                320       
                                                                 
 re_lu (ReLU)                (None, 32)                0         
                                                                 
 dense_1 (Dense)             (None, 32)                1056      
                                                                 
 re_lu_1 (ReLU)              (None, 32)                0         
                                                             

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: `np.unicode` is a deprecated alias for `np.compat.unicode`. To silence this warning, use `np.compat.unicode` by itself. In the likely event your code does not need to work on Python 2 you can use the builtin `str` for which `np.compat.unicode` is itself an alias. Doing this will not modify any behaviour and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  


16/16 [==============================] - 1s 2ms/step


array([[0.25022265],
       [0.96676344],
       [0.16473292],
       [0.9236901 ],
       [0.2136606 ],
       [0.63005537],
       [0.20628439],
       [0.7653687 ],
       [0.67428654],
       [0.27761105],
       [0.5912477 ],
       [0.33972102],
       [0.67899245],
       [0.2664869 ],
       [0.29663262],
       [0.17810853],
       [0.55513674],
       [0.87535924],
       [0.5843467 ],
       [0.19847606],
       [0.99694556],
       [0.19941983],
       [0.17302673],
       [0.49533063],
       [0.99206495],
       [0.20161688],
       [0.05492683],
       [0.9617455 ],
       [0.92790306],
       [0.19840583],
       [0.5046851 ],
       [0.09412075],
       [0.5089855 ],
       [0.19183867],
       [0.666005  ],
       [0.17079595],
       [0.38025293],
       [0.56713474],
       [0.637302  ],
       [0.77338517],
       [0.9763142 ],
       [0.664979  ],
       [0.92973274],
       [0.85237014],
       [0.28870764],
       [0.19593471],
       [0.81294185],
       [0.256

## Reference
[StructuredDataClassifier](/structured_data_classifier),
[AutoModel](/auto_model/#automodel-class),
[StructuredDataBlock](/block/#structureddatablock-class),
[DenseBlock](/block/#denseblock-class),
[StructuredDataInput](/node/#structureddatainput-class),
[ClassificationHead](/block/#classificationhead-class),
[CategoricalToNumerical](/block/#categoricaltonumerical-class).
